In [ ]:
# Importations nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import logging
import os
import sys

# Configuration du logger (pour s'assurer que les messages DEBUG sont visibles)
# Ajouter le répertoire src au path si ce script est exécuté seul
sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'src'))
from adan_trading_bot.common.utils import get_logger
logger = get_logger(__name__)
logger.setLevel(logging.DEBUG) # S'assurer que les messages DEBUG sont traités

logger.info("Notebook d'analyse initialisé.")

In [ ]:
# Chargement des Données
# Assurez-vous que le chemin vers votre fichier de logs est correct
log_file_path = "/home/morningstar/Documents/trading/ADAN/logs/endurance_metrics.jsonl" 
# Remplacez par le chemin réel de votre fichier de logs d'entraînement long

try:
    df_logs = pd.read_json(log_file_path, lines=True)
    logger.info(f"Données chargées depuis {log_file_path}. Nombre d'entrées : {len(df_logs)}")
    logger.debug(f"Aperçu des données :\n{df_logs.head()}")
except FileNotFoundError:
    logger.error(f"Fichier de logs non trouvé : {log_file_path}. Veuillez vérifier le chemin.")
    df_logs = pd.DataFrame() # Créer un DataFrame vide pour éviter les erreurs ultérieures
except Exception as e:
    logger.error(f"Erreur lors du chargement du fichier de logs : {e}")
    df_logs = pd.DataFrame()

In [ ]:
# Préparation des données pour les graphiques
if not df_logs.empty:
    # Convertir les timestamps en datetime si ce n'est pas déjà fait
    if 'timestamp' in df_logs.columns:
        df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'])
        df_logs.set_index('timestamp', inplace=True)
        df_logs.sort_index(inplace=True)

    # Extraire les métriques pertinentes
    # Assurez-vous que ces colonnes existent dans votre fichier JSONL
    metrics_to_plot = ['sl_pct', 'tp_pct', 'portfolio_value', 'drawdown', 'reward_boost', 'penalty_inaction', 'risk_mode']
    for metric in metrics_to_plot:
        if metric not in df_logs.columns:
            logger.warning(f"La métrique '{metric}' n'est pas trouvée dans les données. Elle ne sera pas tracée.")
    
    # Filtrer les données pour les graphiques si nécessaire
    # df_filtered = df_logs.dropna(subset=['sl_pct', 'tp_pct', 'portfolio_value'])
    df_filtered = df_logs # Pour l'instant, utilisons toutes les données chargées

    logger.info("Données préparées pour l'analyse.")
else:
    logger.warning("Aucune donnée à préparer car le DataFrame est vide.")

In [ ]:
# Graphique 1 : Évolution de SL/TP et Capital
if not df_filtered.empty and 'sl_pct' in df_filtered.columns and 'tp_pct' in df_filtered.columns and 'portfolio_value' in df_filtered.columns:
    fig, ax1 = plt.subplots(figsize=(14, 7))

    color = 'tab:red'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('SL/TP (%)', color=color)
    ax1.plot(df_filtered.index, df_filtered['sl_pct'] * 100, color=color, label='Stop Loss (%)')
    ax1.plot(df_filtered.index, df_filtered['tp_pct'] * 100, color='tab:orange', label='Take Profit (%)')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend(loc='upper left')

    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('Valeur du Portefeuille', color=color)  
    ax2.plot(df_filtered.index, df_filtered['portfolio_value'], color=color, label='Valeur du Portefeuille')
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend(loc='upper right')

    fig.tight_layout()  
    plt.title('Évolution du Stop Loss, Take Profit et Valeur du Portefeuille')
    plt.grid(True)
    plt.show()
else:
    logger.warning("Données insuffisantes pour le Graphique 1.")

In [ ]:
# Graphique 2 : Évolution du Drawdown avec indication du Risk Mode
if not df_filtered.empty and 'drawdown' in df_filtered.columns and 'risk_mode' in df_filtered.columns:
    fig, ax = plt.subplots(figsize=(14, 7))

    ax.plot(df_filtered.index, df_filtered['drawdown'] * 100, color='tab:purple', label='Drawdown (%)')
    ax.set_xlabel('Date')
    ax.set_ylabel('Drawdown (%)')
    ax.set_title('Évolution du Drawdown et Modes de Risque')
    ax.grid(True)

    # Colorier les zones où risk_mode est 'DEFENSIVE'
    defective_periods = df_filtered[df_filtered['risk_mode'] == 'DEFENSIVE']
    for start, end in zip(defective_periods.index, defective_periods.index[1:]):
        if (end - start).total_seconds() < 60: # Éviter de colorier des points isolés
            continue
        ax.axvspan(start, end, color='red', alpha=0.2, label='Mode Défensif' if 'Mode Défensif' not in [l.get_label() for l in ax.lines] else "")

    ax.legend()
    plt.show()
else:
    logger.warning("Données insuffisantes pour le Graphique 2.")

In [ ]:
# Histogramme 3 : Distribution des reward_boost
if not df_filtered.empty and 'reward_boost' in df_filtered.columns:
    plt.figure(figsize=(10, 6))
    plt.hist(df_filtered['reward_boost'].dropna(), bins=30, color='skyblue', edgecolor='black')
    plt.title('Distribution des Reward Boost')
    plt.xlabel('Valeur du Reward Boost')
    plt.ylabel('Fréquence')
    plt.grid(axis='y', alpha=0.75)
    plt.show()
else:
    logger.warning("Données insuffisantes pour l'Histogramme 3.")

In [ ]:
# Histogramme 4 : Distribution des penalty_inaction
if not df_filtered.empty and 'penalty_inaction' in df_filtered.columns:
    plt.figure(figsize=(10, 6))
    plt.hist(df_filtered['penalty_inaction'].dropna(), bins=30, color='lightcoral', edgecolor='black')
    plt.title('Distribution des Pénalités d'Inaction')
    plt.xlabel('Valeur de la Pénalité d'Inaction')
    plt.ylabel('Fréquence')
    plt.grid(axis='y', alpha=0.75)
    plt.show()
else:
    logger.warning("Données insuffisantes pour l'Histogramme 4.")